### transformers for text processing
- speed
- understand the relationship between words regardless of the distance
- human like texts

### transformers components
- Encoder: process input data
- Decoder: reconstructs the output
- feed forward networks: refine understanding
- positional encodings: ensure order matters
- multi head attention: captures multiple inputs or sentiments